In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
epsilon = 0.0001
alpha = 2
A = 200
B = 10
N = 10
scale = 100
h = 0.01 #step

all_points = [np.asarray([np.random.rand()*scale, np.random.rand()*scale, np.random.rand(), np.random.rand()]) \
                                                                                  for _ in np.arange(N) ]

coordinates_of_all_points = np.random.rand(N, 2)*scale
velocities_of_all_points = np.random.rand(N, 2)
print("Coordinates of all points:\n", coordinates_of_all_points, \
      "\n\nVelocities of all points:\n", velocities_of_all_points)


Coordinates of all points:
 [[88.44459581 50.38695868]
 [60.9351533  17.85486325]
 [38.88222804 23.78133058]
 [25.79572241  3.07743279]
 [56.11765699 44.5568347 ]
 [83.73328787  8.94461887]
 [78.21496727 11.78515066]
 [83.35879507 58.61480187]
 [48.59965432 25.55860174]
 [84.48135507 95.12038439]] 

Velocities of all points:
 [[0.94036655 0.46441927]
 [0.27915574 0.04522994]
 [0.94426314 0.73752488]
 [0.24389162 0.57994901]
 [0.93631547 0.31223465]
 [0.2201051  0.20065754]
 [0.29463352 0.43425015]
 [0.58529328 0.48614194]
 [0.77514799 0.63091386]
 [0.60028729 0.23082984]]


In [3]:
# def new_coordinates()

In [4]:
def force_of_interaction(velocity1, point_ID1, point_ID2, delta_time):
    old_coord_diff = coordinates_of_all_points[point_ID2]- coordinates_of_all_points[point_ID1]
    old_velocity_diff = velocities_of_all_points[point_ID2]- velocity1
    new_coord_diff = old_velocity_diff * delta_time + old_coord_diff
    return new_coord_diff/np.power(np.linalg.norm(new_coord_diff), alpha)

In [5]:
def find_acceleration(point_ID, delta_velocity, delta_time):
    velocity = velocities_of_all_points[point_ID] + delta_velocity
    return (   A-B*np.power(np.linalg.norm(velocity), 2)   )*velocity+\
            np.sum([force_of_interaction(velocity, point_ID, body_ID, delta_time) if body_ID != point_ID else [0, 0] for body_ID in np.arange(N)], axis = 0)

In [6]:
def k_1_computation(point_ID):
    return h*find_acceleration(point_ID, 0, 0)

In [7]:
def k_2_computation(point_ID):
    return h*find_acceleration(point_ID, k_1_computation(point_ID), h)

In [8]:
def update_velocity(point_ID):
    return velocities_of_all_points[point_ID] + (k_1_computation(point_ID) + k_2_computation(point_ID))/2

In [26]:
velocities_of_all_points = np.asarray([update_velocity(point_ID) for point_ID in np.arange(N)])
coordinates_of_all_points += velocities_of_all_points * h

In [25]:
a = np.array([[0, 1, 2, 3],[4, 5, 6, 7]])
np.asarray(velocities_of_all_points).shape

(10, 2)

In [28]:
print("Coordinates of all points:\n", coordinates_of_all_points, \
      "\n\nVelocities of all points:\n", velocities_of_all_points)


Coordinates of all points:
 [[88.41277295 50.37133659]
 [60.90942267 17.85067357]
 [38.89232435 23.78894648]
 [25.79556409  3.07705219]
 [56.10984231 44.55421406]
 [83.70686863  8.92036146]
 [78.21883085 11.79087162]
 [83.37056611 58.62457376]
 [48.5845845  25.546323  ]
 [84.48316686 95.1210786 ]] 

Velocities of all points:
 [[-3.18228531 -1.56220871]
 [-2.57306257 -0.41896834]
 [ 1.00963048  0.76158955]
 [-0.01583246 -0.03806013]
 [-0.78146789 -0.26206423]
 [-2.64192354 -2.42574108]
 [ 0.38635858  0.57209672]
 [ 1.17710443  0.97718908]
 [-1.50698148 -1.22787397]
 [ 0.18117842  0.06942135]]
